In [136]:
import ee
import geemap
import os
import json
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

In [137]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [138]:
file_path = os.path.abspath('C:\\Users\\jfsdi\\Documents\\asturias.json')
with open(file_path) as f:
    json_data = json.load(f)

In [139]:
json_layer = GeoJSON(data=json_data, name='US States JSON', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
Map.add_layer(json_layer)

ee_data = geojson_to_ee(json_data)
Map.addLayer(ee_data, {}, "roi")


In [140]:
start_date = '2020-08-01'
end_date = '2020-12-31'

l8 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(ee_data) \
    .filterDate(start_date, end_date)

In [141]:
print(l8.size().getInfo())

235


In [142]:
median = l8.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')

In [143]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [144]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [145]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [146]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

In [147]:
withNDVI = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY)

In [ ]:
greenest = withNDVI.qualityMosaic('NDVI')

In [148]:
greenest.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy']

In [149]:
ndvi = greenest.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [150]:
Map.addLayer(greenest, visParams, 'Greenest pixel')
Map

Map(bottom=24195.0, center=[43.644025847699496, -4.394531250000001], controls=(WidgetControl(options=['positio…

In [151]:
Map.addLayer(greenest.select('month'), {'palette': ['red', 'blue'], 'min': 1, 'max': 12}, 'Greenest month')


In [152]:
Map.addLayer(greenest.select('doy'), {'palette': ['brown', 'green'], 'min': 1, 'max': 365}, 'Greenest doy')

In [153]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'landsat.tif')

In [154]:
image = greenest.select('doy')
geemap.ee_export_image(image, filename=filename, scale=90, region=ee_data, file_per_band=True)

Generating URL ...
An error occurred while downloading.
Image.clipToBoundsAndScale, argument 'geometry': Invalid type.
Expected type: Geometry.
Actual type: FeatureCollection.
